In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
import implicit
from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

/home/user_3/medistream-recsys/Script


# Load Data

In [31]:
df_book = pd.read_json('/home/user_1/medistream-recsys/Script/YS/df_book_clean.json')
df_book = df_book.reset_index()

In [32]:
df_book['book_corpus_str'] = df_book['book_corpus_konlp'].apply(' '.join)
df_book

,index,_id,category_id_x,date_created,name_x,description,meta_description_x,meta_title,tags_x,three_months,regular_price_y,m_name,L_name,s_name,book_corpus,book_corpus_konlp,book_corpus_str
0,0,5d13115e32026c0b35383897,5cf8bbba0098b2225c5dfaa3,{'$date': '2019-06-26T06:31:58.090Z'},KCD 한방내과 진찰진단 가이드라인,의 번째 큐레이션 상시판매 권승원의 한방내과 진찰진단 가이드라인 의학은빠른속...,진료에 어려움 있을 때 먼저 찾는 책,KCD 한방내과 진찰진단 가이드라인,[가온해미디어],35,150000.0,전문,도서,[가온해미디어],"KCD 한방내과 진찰진단 가이드라인, 의 번째 큐레이션 상시판매 권승원의 한방내과...","[병력, 고려, 의, 더, 집필, 추천사, 지구별, 소변, 이해, 외, 호평, 환자...",병력 고려 의 더 집필 추천사 지구별 소변 이해 외 호평 환자 근래 때 한의사 줄 ...
1,1,5d9edfe70dabe405b156e213,5cf8bbba0098b2225c5dfaa3,{'$date': '2019-10-10T07:38:15.467Z'},우리 아이 감기,우리 아이 감기부제 약물 남용과 과잉치료가 없는 새로운 감기 진료를 목표로 ...,약물남용과 과잉치료가 없는 새로운 감기 진료를 목표로,우리 아이 감기,[물고기숲],7,NaN,None,None,None,"우리 아이 감기,우리 아이 감기부제 약물 남용과 과잉치료가 없는 새로운 감기 진료를...","[다시, 약, 기관, 신장염, 관련, 보균, 육아, 보시, 활용, 공포증, 그, 중...",다시 약 기관 신장염 관련 보균 육아 보시 활용 공포증 그 중이 생각 소아 정리 부...
2,2,5e3cb2904267e105dfff13b3,5cf8bbba0098b2225c5dfaa3,{'$date': '2020-02-07T00:42:56.629Z'},통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,_ 출판사 리뷰년전 소수 인원이 모여 시작한 통증 공부 모임이 작은 불씨가 ...,하이드로다이섹션: 포도당을 이용한 신경의 유착박리와 신경주변 환경개선,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,[메디안북],2,190000.0,전문,도서,[메디안북],"통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉, _ 출판사 리뷰년전 소수...","[통해, 요추, 절, 모임, 목차, 접근, 주야, 걸음, 소속, 복신, 의, 그동안...",통해 요추 절 모임 목차 접근 주야 걸음 소속 복신 의 그동안 더 관계 길 불철 무...
3,3,5e3d06b64267e105dfff13e0,5cf8bbba0098b2225c5dfaa3,{'$date': '2020-02-07T06:41:58.622Z'},의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),의사들이 꼭 알고 있어야 하는 응급의학 실전지침서,,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),[메디안북],0,130000.0,전문,도서,[메디안북],"의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),의사들이 꼭 알고 있어야...","[응급의학, 실전, 지침, 꼭, 알, 의사]",응급의학 실전 지침 꼭 알 의사
4,4,5e3d0da94267e105dfff13ed,5cf8bbba0098b2225c5dfaa3,{'$date': '2020-02-07T07:11:37.566Z'},복부 초음파 정석,_ 저자서문 초음파 검사는 임상 영역에서 청진기로 불릴 정도로 간편한...,정확한 복부초음파를 위한 필독서,복부 초음파 정석,[메디안북],1,60000.0,전문,도서,[메디안북],"복부 초음파 정석, _ 저자서문 초음파 검사는 임상 영역에서 청진기로 ...","[췌, 횡단, 진수, 이유, 이해, 면역성, 지방, 수고, 소견, 대비, 모습, 난...",췌 횡단 진수 이유 이해 면역성 지방 수고 소견 대비 모습 난관 번역 해부 상하 병...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,370,63182f4a6c2e37002369d6b9,5cf8bbba0098b2225c5dfaa3,{'$date': '2022-09-07T05:42:34.413Z'},임상족부학과 족부보조기,저자 서문중부족한 실력이지만 여기저기 다니며 족부학에 대해 강의하다 보니 마지막...,,임상족부학과 족부보조기,[대한의학서적],0,70000.0,전문,도서,[대한의학서적],"임상족부학과 족부보조기, 저자 서문중부족한 실력이지만 여기저기 다니며 족부학에 대...","[족부학, 약, 술기, 목차, 접근, 한강, 거기, 설명, 고려, 세월, 더, 머리...",족부학 약 술기 목차 접근 한강 거기 설명 고려 세월 더 머리 그동안 제일 걸음 이...
368,371,63182f4b6c2e37002369d6ba,5cf8bbba0098b2225c5dfaa3,{'$date': '2022-09-07T05:42:35.702Z'},기능의학적 시각으로 비만관리 시작하기,저자 서문중지구상의 인구 중에 이 비만이라니 정말 놀라운 일이다 아마 시간이 흐...,비만관리를 위한 새로운 각도의 접근법,기능의학적 시각으로 비만관리 시작하기,[대한의학서적],0,40000.0,전문,도서,[대한의학서적],"기능의학적 시각으로 비만관리 시작하기, 저자 서문중지구상의 인구 중에 이 비만이라...","[상담, 목차, 접근, 시대, 거나, 바탕, 일이, 의, 더, 좌절, 환자, 해답,...",상담 목차 접근 시대 거나 바탕 일이 의 더 좌절 환자 해답 때 강요 하나로 관련 ...
369,372,63182f4c6c2e37002369d6bb,5cf8bbba0098b2225c5dfaa3,{'$date': '2022-09-07T05:42:36.934Z'},일차진료 Clinical Manual 7판,일차진료 판 일차진료 판본 매뉴얼은 년 「영동 매뉴얼」이라는 제목의...,일차진료 클리니컬 매뉴얼,일차진료 Clinical Manual 7판,[대한의학서적],0,50000.0,경영,도서,[대한의학서적],"일차진료 Clinical Manual 7판, 일차진료 판 일차진료 판본...","[비염, 뇌하수체, 딸꾹질, 관련, 경전, 혈, 뇌혈관, 통풍, 병변, 활용, 그,...",비염 뇌하수체 딸꾹질 관련 경전 혈 뇌혈관 통풍 병변 활용 그 기술 중이 심부전 혈...
370,373,63182f4d6c2e37002369d6bc,5cf8bbba0098b2225c5dfaa3,{'$date': '2022-09-07T05:42:37.928Z'},醫學心悟(의학심오) 톺아보기,고전에서 배우는 한의치료 오뒷세이아① 《醫學心悟는 청대 초...,고전에서 배우는 한의치료 오뒷세이아①,醫學心悟(의학심오) 톺아보기,[물고기숲],1,45000.0,전문,도서,[물고기숲],"醫學心悟(의학심오) 톺아보기, 고전에서 배우는 한의치료 오뒷세이...","[통해, 도전, 무력, 질, 목차, 치법, 시대, 바탕, 발열, 의, 것이므, 더,...",통해 도전 무력 질 목차 치법 시대 바탕 발열 의 것이므 더 안과 무엇 소변 초 소...


In [33]:
article_sum  = pd.read_json('/home/user_4/CBF/article_sum.json')
article_sum = article_sum.rename(columns={"meaningful unit": "meaningful_unit_list"})
article_sum['meaningful_unit_str'] = article_sum['meaningful_unit_list'].apply(lambda x: ' '.join(x))
article_sum

,id,title,subtitle,content_tag_removed,meaningful_unit_list,meaningful_unit_str
0,43,디스크탈출증 재흡수 케이스,"F/30대 후반, R/O Herniated Disc L5-S1, Extrusion","케이스의 종류 치료성공 케이스 환자 성별, 나이 여자 / 30대 후반 ...","[의, 본원, 좌측, 계획, 요통, 저하, 다리, 내용, 반응, 봉침, 추간판, 통...",의 본원 좌측 계획 요통 저하 다리 내용 반응 봉침 추간판 통점 정상 주사 이상근 ...
1,61,여기에 질문하는거 맞나요?,None,수고하십니다. 자보환자가 한약 치료 끝나고 보험제제를 계속 원하십니다. 저희 ...,"[수고, 보험, 제, 엑스, 저희, 한약, 자보, 병원, 제품, 인정, 치료, 제제...",수고 보험 제 엑스 저희 한약 자보 병원 제품 인정 치료 제제 뭐 환자 라시 계속
2,66,뒷목통증 좌측방사통을 주소로 내원한 환자,,"케이스의 종류 치료 성공 케이스 환자 성별, 나이 남자 /...","[좌측, 압통, 계획, 저하, 내용, 반응, 정상, 주사, 손가락, 우측, 발살바,...",좌측 압통 계획 저하 내용 반응 정상 주사 손가락 우측 발살바 증상 음주 학적 상기...
3,46,퇴행성 관절염 환자 무릎에 물이 찬 경우.,"물이 차는 원인과, 어느 공간에 차는 것이며, 그 물이란 것의 정체는 무엇인가요.",한의원에서 대응해서 할 수 있는 치료주사기로 빼야 하는지 등는 무엇이 있나요?,"[무엇, 치료, 등, 주사기, 수, 대응, 한의원]",무엇 치료 등 주사기 수 대응 한의원
4,51,비전형적인 Cauda equina syn.,"F/36, R/O Cauda equina syn.","케이스의 종류 함께 논의하고 싶은 케이스 환자 성별, 나이 ...","[의, 좌측, 계획, 추가, 로, 확인, 저하, 바로, 다리, 뒤, 담, 전화, 정...",의 좌측 계획 추가 로 확인 저하 바로 다리 뒤 담 전화 정상 백병원 충격 우측 증...
...,...,...,...,...,...,...
3433,53555,"[나의 인턴 일지] Ep.04 한의대생, 퇴사를 했다.",그리고 독일에 왔다.,선생님들 안녕하세요! 꽤 오랜만에 글을 씁니다. 저 퇴사를 했습...,"[취재, 심지어, 인사, 다시, 계획, 사실, 바로, 불, 실질, 사적, 양산, 현...",취재 심지어 인사 다시 계획 사실 바로 불 실질 사적 양산 현장 빈도 집 첫 정상 ...
3434,53850,초음파 케이스: shoulder impingement synd. (SASD burs...,case 13,# CASE 13 houlder iigeet yd. SASD uriti 37...,"[내강, 때문, 의, 막, 좌측, 구급대원, 화살표, 상근, 세, 빨간색, 쉬, 어...",내강 때문 의 막 좌측 구급대원 화살표 상근 세 빨간색 쉬 어깨 비교 외 계심 환자...
3435,53945,비판적으로 논문 읽기: 통계적 보정에 대해,통계적으로 보정된 값은 어떻게 해석하는가,"얼마 전 COVID-19를 앓고 난 뒤 남아 있는 여러 증상들, 롱 코비드Log ...","[그림, 의, 관계, 막, 폐쇄, 관점, 다시, 연구자, 추가, 표현, 관심, 확인...",그림 의 관계 막 폐쇄 관점 다시 연구자 추가 표현 관심 확인 인과 사실 좀 질병 ...
3436,54190,[작가가 입사했다] Ep.09 작가가.. 혹사했다..,None,"여름 휴가 기간은 쏜살같이 지났고, 여전히 메디스트림은 바쁘다. 엎친데 덮...","[연구원, 이번, 깨알, 폰임, 도전, 다시, 야근, 계획, 다, 관심, 피신, 최...",연구원 이번 깨알 폰임 도전 다시 야근 계획 다 관심 피신 최근 고문 사실 좀 오픈...


# Concat Data

In [34]:
df_book.rename(columns={'_id':'id','name_x':'title','book_corpus_str':'corpus'},inplace=True)
article_sum.rename(columns={'meaningful_unit_str':'corpus'},inplace=True)

In [35]:
df_concat = pd.concat([df_book[['id','title','corpus']],article_sum[['id','title','corpus']]]).rename(columns={0:'corpus'}).reset_index()

In [36]:
df_concat

,index,id,title,corpus
0,0,5d13115e32026c0b35383897,KCD 한방내과 진찰진단 가이드라인,병력 고려 의 더 집필 추천사 지구별 소변 이해 외 호평 환자 근래 때 한의사 줄 ...
1,1,5d9edfe70dabe405b156e213,우리 아이 감기,다시 약 기관 신장염 관련 보균 육아 보시 활용 공포증 그 중이 생각 소아 정리 부...
2,2,5e3cb2904267e105dfff13b3,통증치료를 위한 신경초음파와 하이드로다이섹션 테크닉,통해 요추 절 모임 목차 접근 주야 걸음 소속 복신 의 그동안 더 관계 길 불철 무...
3,3,5e3d06b64267e105dfff13e0,의사들이 꼭 알고 있어야 하는 응급의학 실전지침서(1-2),응급의학 실전 지침 꼭 알 의사
4,4,5e3d0da94267e105dfff13ed,복부 초음파 정석,췌 횡단 진수 이유 이해 면역성 지방 수고 소견 대비 모습 난관 번역 해부 상하 병...
...,...,...,...,...
3805,3433,53555,"[나의 인턴 일지] Ep.04 한의대생, 퇴사를 했다.",취재 심지어 인사 다시 계획 사실 바로 불 실질 사적 양산 현장 빈도 집 첫 정상 ...
3806,3434,53850,초음파 케이스: shoulder impingement synd. (SASD burs...,내강 때문 의 막 좌측 구급대원 화살표 상근 세 빨간색 쉬 어깨 비교 외 계심 환자...
3807,3435,53945,비판적으로 논문 읽기: 통계적 보정에 대해,그림 의 관계 막 폐쇄 관점 다시 연구자 추가 표현 관심 확인 인과 사실 좀 질병 ...
3808,3436,54190,[작가가 입사했다] Ep.09 작가가.. 혹사했다..,연구원 이번 깨알 폰임 도전 다시 야근 계획 다 관심 피신 최근 고문 사실 좀 오픈...


# TF-IDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
#Define a TF-IDF Vectorizer Object
tfidf = TfidfVectorizer()

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_concat['corpus']) # fit은 training data에 사용?

#Output the shape of tfidf_matrix
print(tfidf_matrix.shape)

(3810, 29825)


In [39]:
tfidf_dict = tfidf.get_feature_names()
data_array = tfidf_matrix.toarray()
data = pd.DataFrame(data_array, columns=tfidf_dict)
data

/home/user_1/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,가가,가감,가감례,가감법,가건물,가게,가격,가격표,가결,가계,가계도,가계비,가고시마,가공,가공업,가구,가글,가금,가금류,가급,가기,가까스로,가까이,가끔,가나,가나다,가나다라,가나다순,가나타,가년,가늠,가늠자,가능,가능성,가다가,가다랑어,가닥,가대,가도,가독성,가돌리늄,가동,가동성,가동역,가드,가득,가든,가디스,가디언,가뜩이나,...,흰색,흰자,흰자위,흰점박이꽃무지,힁단,힁단보,히노,히데,히드로,히든카드,히로,히로시마,히르쉬스프룽,히르쉬스프룽병,히르슈슈프룽병,히스,히스타민,히스테리,히스토리,히스톤,히스티딘,히알루,히알루론,히알루론산,히알루론주사,히어로,히요스엽,히요우,히카루,히터,히트,히팅블럭,히포크라테스,힌국,힌의학,힌트,힐난,힐러,힐링,힐색힐,힐앤,힐튼,힘껏,힘베,힘살,힘줄,힘폐쇄장,힙니,힙업,힙업밴드
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.055919,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.090164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.062574,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10975,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3805,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.069425,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3806,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3807,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0

# 책과 아티클 사이 코사인 유사도 구하기

In [40]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
book_article_cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# book_article_cosine_sim = cosine_similarity(book_tfidf_matrix, article_tfidf_matrix)
print('코사인 유사도 연산 결과 :',book_article_cosine_sim.shape)

코사인 유사도 연산 결과 : (3810, 3810)


In [41]:
# book_to_index = dict(zip(df_book['name_x'], df_book.index))
# article_to_index = dict(zip(article_sum['title'], article_sum.index))
title_to_index = dict(zip(df_concat['title'],df_concat.index))
id_to_index = dict(zip(df_concat['id'],df_concat.index))

In [42]:
def get_recommendations(title, cosine_sim=book_article_cosine_sim):
    # 선택한 책,아티클의 타이틀로부터 해당 책,아티클의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 책과 모든 책,아티클과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 책들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 책을 받아온다.
    sim_scores = sim_scores[1:101]

    # 가장 유사한 10개의 책의 인덱스를 얻는다.
    book_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 책의 제목을 리턴한다.
    return df_concat['title'].iloc[book_indices]
#     return sim_scores

In [43]:
def get_recommendations_books(id, cosine_sim=book_article_cosine_sim):
    # 선택한 책의 타이틀로부터 해당 책의 인덱스를 받아온다.
    idx = id_to_index[id]

    # 해당 책과 모든 책,아티클과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 책,아티클들을 정렬한다.
    sim_scores = sorted(sim_scores[:372], key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 책을 받아온다.
    sim_scores = sim_scores[1:101]

    # 가장 유사한 10개의 책의 인덱스를 얻는다.
    book_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 책의 제목을 리턴한다.
    return df_concat['id'].iloc[book_indices]

In [44]:
def get_recommendations_articles(title, cosine_sim=book_article_cosine_sim):
    # 선택한 책의 타이틀로부터 해당 책의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 책과 모든 책,아티클과의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 아티클들을 정렬한다.
    sim_scores = sorted(sim_scores[372:], key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 책을 받아온다.
    sim_scores = sim_scores[1:101]

    # 가장 유사한 10개의 책의 인덱스를 얻는다.
    book_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 책의 제목을 리턴한다.
    return df_concat['title'].iloc[book_indices]

In [45]:
get_recommendations_articles('피부질환의 약물과 치료',cosine_sim=book_article_cosine_sim)

2982                       [책소개] 『침의 과학적 접근의 이해』 - 옮김 이승훈
1533                                             한의학과 EBM
2610                  리바운드 현상[Rebound phenomenon]진료의 첫 걸음
1550                        코로나바이러스감염증-19  한의진료 권고안  제 1판
2474                           건선[Psoriasis] 피부진료의 첫걸음(1)
2146                                       항염증제/오타코이드: 개요
942                                     유전자의 이름은 어떻게 만드나?
2173                    Volume effect, 그리고 Washing effect
2513                       리바운드 현상[Rebound phenomenon](2)
2057                                          알레르기 비염·화분증
2357                               '한방내과'가 궁금한 분들을 위한 인터뷰
1521                             신종코로나바이러스폐렴 중국 지침 제5판 해설
2920                              [대만드] 경희대학교 조기호 교수님 인터뷰
2990                                  귀출파징탕은 實證어혈성 월경통 처방
891                                       생리학을 공부해야 할 필요성
1720                                      황금탕(PHY906)의 도전
1062                       포비돈-요오드 용액(베타딘)과 관련하여 생각해볼 내용들
3800          

# Order 기록이 있으면 불러오기 최신성 반영
-> 가장 마지막에 산 도서를 기준으로 추천

In [46]:
df_order = pd.read_json('/fastcampus-data/select_column_version_4.json')

In [47]:
date_state = "2022-05-12"

# paid orders만 가져오기
df_order['date_paid'] = pd.to_datetime(df_order['date_paid'])
df_only_paid = df_order[~df_order['date_paid'].isna()]
# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > date_state]
# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
# 도서 카테고리만 가져오기
only_book = complete_df[complete_df['name'] == '도서']

# 유저가 중복으로 아이템 구매 삭제
df_duplicated_book = only_book.drop_duplicates(subset=['customer_id','product_ids'])
df_order_book = df_duplicated_book.sort_values(by='date_paid').reset_index(drop=True)

In [48]:
# df_last_order = df_order_book.drop_duplicates(subset='customer_id',keep='last')
df_order_book

,_id,date_created,regular_price,sale_price,three_months,date_paid,customer_id,paid,name_x,category_id_y,product_ids,quantity,price,price_total,age_group,한의사 여부,사업자 여부,cancelled,name,slug
0,627c50279d9388002406326a,2022-03-24T01:25:28.483Z,40000.0,38800.0,26.0,2022-05-12 00:10:00.012000+00:00,5d838c7f3f0e6805c4706894,True,장골의 PI 변위는 없다,5cf8bbba0098b2225c5dfaa3,623bc8886f766b0024668eb5,1.0,38800.0,38800.0,30-39,1.0,0.0,False,도서,book
1,627c519c9d93880024063275,2022-04-28T08:08:41.248Z,210000.0,199500.0,0.0,2022-05-12 00:15:59.124000+00:00,5ebc8bee09982e0735b2d835,True,초음파 가이드 근골격계 통증 치료의 정석,5cf8bbba0098b2225c5dfaa3,626a4b89e1579900234bd4af,1.0,199500.0,199500.0,30-39,1.0,1.0,False,도서,book
2,627c51ca9d93880024063280,2022-04-28T08:08:41.248Z,210000.0,199500.0,0.0,2022-05-12 00:16:31.210000+00:00,5d931d770dabe405b156d63b,True,초음파 가이드 근골격계 통증 치료의 정석,5cf8bbba0098b2225c5dfaa3,626a4b89e1579900234bd4af,1.0,199500.0,199500.0,30-39,1.0,1.0,False,도서,book
3,627c51c69d9388002406327e,2022-04-28T08:08:41.248Z,210000.0,199500.0,0.0,2022-05-12 00:17:17.201000+00:00,5f6be49c48060c52caf1528e,True,초음파 가이드 근골격계 통증 치료의 정석,5cf8bbba0098b2225c5dfaa3,626a4b89e1579900234bd4af,1.0,199500.0,199500.0,50-59,1.0,1.0,False,도서,book
4,627c51de9d93880024063289,2022-04-28T08:08:41.248Z,210000.0,199500.0,0.0,2022-05-12 00:17:28.593000+00:00,5daf9e520dabe405b156f082,True,초음파 가이드 근골격계 통증 치료의 정석,5cf8bbba0098b2225c5dfaa3,626a4b89e1579900234bd4af,1.0,199500.0,199500.0,40-49,1.0,1.0,False,도서,book
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,63202f736c2e37002369f06e,2021-10-28T05:52:44.081Z,65000.0,58500.0,52.0,2022-09-13 07:22:18.847000+00:00,5dee019d4267e105dffef02d,True,트리거포인트 침치료,5cf8bbba0098b2225c5dfaa3,617a3aacbc63410023471ab6,1.0,58500.0,58500.0,20-29,1.0,0.0,False,도서,book
5906,632033be6c2e37002369f09a,2022-04-12T06:34:15.781Z,30000.0,28500.0,11.0,2022-09-13 07:40:23.148000+00:00,617e903bbc6341002347262a,True,스파이랄 및 키네지오 테이핑,5cf8bbba0098b2225c5dfaa3,62551d67cb28e6002469d8fb,1.0,28500.0,28500.0,None,1.0,1.0,False,도서,book
5907,63203ac86c2e37002369f0e7,2020-02-11T02:48:24.348Z,55000.0,49500.0,51.0,2022-09-13 08:15:51+00:00,6136d8e597686100197d7db2,True,소아피부질환해설,5cf8bbba0098b2225c5dfaa3,5e4215f84267e105dfff17b9,1.0,49500.0,49500.0,40-49,1.0,1.0,False,도서,book
5908,632024716c2e37002369f008,2019-12-09T02:48:00.845Z,40000.0,36000.0,21.0,2022-09-13 08:51:40+00:00,5e5f01c7bfe4260944782d10,True,한의 피부진료 첫 걸음,5cf8bbba0098b2225c5dfaa3,5dedb5e04267e105dffeef43,1.0,36000.0,36000.0,30-39,1.0,1.0,False,도서,book


# Train_test

In [49]:
from datetime import datetime, timedelta

date = df_order_book['date_paid'].max()-timedelta(weeks=3)
train_before_preprocess = df_order_book[df_order_book['date_paid'] < date]
test_before_preprocess = df_order_book[df_order_book['date_paid'] >= date]

# train 1개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts().index
train = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]
train = train.reset_index()

In [50]:
# 전처리할 test index 확인
drop_row_index = test_before_preprocess['customer_id'].value_counts().index

test_list = []
for idx in drop_row_index:
    test_list.append(test_before_preprocess[test_before_preprocess['customer_id']==idx])
test = pd.concat(test_list)

# train 있는 유저에 대해서만 진행
test = test[test['customer_id'].isin(train['customer_id'])]

In [51]:
train.shape,test.shape

((5024, 21), (464, 20))

In [52]:
train['customer_id'].nunique(),test['customer_id'].nunique()

(2323, 332)

# Modeling

In [53]:
# real test 
ground_trues = []
for user_id in test['customer_id'].unique():
    ground_trues.append({'id': user_id,\
    'items':list(test[test['customer_id']==user_id].product_ids)
    })

In [54]:
ground_trues[0]

{'id': '5ff7fdc7d6491c001b1785f6',
 'items': ['5f9676b2512031633431fa6b',
  '61e777f57c9ea1003fa99d6f',
  '5d59ee854e77525ec5ca1212',
  '6285af5e9d9388002406533f',
  '628ecd6e9d9388002406f05f',
  '629860589d93880024071acb',
  '5e4ccce44267e105dfff1e99',
  '629860569d93880024071aca',
  '610b6f9ad26af90681b1e699',
  '629860599d93880024071acc',
  '62bbee3a08e04900234e36c8',
  '5f167b2d221c8b5f77c8f69d',
  '5e4b6a984267e105dfff1dfe']}

In [55]:
a = ['a','b','c','d']
b = ['a']
[x for x in a if x not in b]

['b', 'c', 'd']

In [85]:
import random
train_recommended = []

# train기간동안 구매한 책을 뽑는다.
for user_id in train['customer_id'].unique():
    train_recommended.append({'id':user_id, \
                             'items':list(train[train['customer_id']==user_id].product_ids)
                             })
# 구매한 책마다 추천 100권씩 비슷한 책 추천 
진행
train_recommended_books = []
for user in train_recommended :
    temp = []
    items = [list(get_recommendations_books(product_id)) for product_id in user['items']]
    if len(items)>1:
        temp = sum(items, [])
    # 추천된 책 중 기존에 산 책은 뺀다    
        items = set([x for x in temp if x not in user['items']])
        items = list(random.sample(items,100))
    else:
        items = items[0]
    train_recommended_books.append({'id':user['id'], \
#                                     'items': sum([list(get_recommendations_books(product_id)) for product_id in user['items']], [])
                                    'items': items
                                   })
    

In [58]:
len(set(train_recommended_books[2]['items']))

100

In [59]:
len(set(train_recommended_books[2]['items']) - set(train_recommended[2]['items']))

100

# Evaluate

In [72]:
class CustomEvaluator:
    # relavence 모두 1로 동일하게 봄
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))
    

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(1000)]
    '''
    idcgs 예시, item 3개 추천되므로 3.074281787960283 가 됩니다.
    [0, 1.4426950408889634, 2.352934267515801, 3.074281787960283]
    '''

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _eval(self, gt_list, rec_list):
        gt_dict = {g["id"]: g for g in gt_list}
        ndcg_score = 0.0

        for rec in rec_list:
            gt = gt_dict[rec["id"]]
            ndcg_score += self._ndcg(gt["items"], rec["items"])


        ndcg_score = ndcg_score / len(rec_list)


        return ndcg_score

    def evaluate(self, gt_list, rec_list):
        try:
            ndcg_score = self._eval(gt_list, rec_list)
            print(f"nDCG: {ndcg_score:.6}")
        except Exception as e:
            print(e)


In [88]:
predict_list = []
for user_id in train_recommended_books:
    if user_id['id'] in list(test['customer_id'].unique()):
        predict_list.append(user_id)
predict_list

[{'id': '5ebc8bee09982e0735b2d835',
  'items': ['5e40cab84267e105dfff155b',
   '5e4b6b724267e105dfff1dff',
   '600e6471527223001a206920',
   '63182f486c2e37002369d6b8',
   '6285af659d93880024065346',
   '60f7d00a09c63a06ed1476c2',
   '625e57080d1b510022676c27',
   '6279c9439d93880024062730',
   '60f4ff2bd26af90681b1cac5',
   '60191de3527223001a207875',
   '62551d67cb28e6002469d8fb',
   '5e4b775a4267e105dfff1e1f',
   '5e40d9f24267e105dfff1565',
   '5f167b2d221c8b5f77c8f69f',
   '6045d81940a8110019dda9ed',
   '5d13115e32026c0b35383897',
   '602cf6532c593a001aceac88',
   '5f167b2b221c8b5f77c8f698',
   '5e4cb11d4267e105dfff1e8d',
   '5f167b26221c8b5f77c8f68e',
   '62cd2c373fc97a002bec850a',
   '5e3d0da94267e105dfff13ed',
   '61d7a0d87560f900324cdaa3',
   '5e42118d4267e105dfff17b0',
   '5d132e964b25b80d9fb1f352',
   '602cf654527223001a209638',
   '5e3d02de4267e105dfff13dd',
   '628ecd709d9388002406f060',
   '6135810d97686100197d7a89',
   '6182113bbc63410023473754',
   '60f7d00609c63a06ed147

In [89]:
# ALS 
evaluator = CustomEvaluator()
evaluator.evaluate(ground_trues, predict_list)

nDCG: 0.0969341
